In [1]:
!pip install pyspark


     ---------------------------------------- 0.0/317.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/317.0 MB ? eta -:--:--
     -------------------------------------- 0.0/317.0 MB 131.3 kB/s eta 0:40:15
     -------------------------------------- 0.0/317.0 MB 219.4 kB/s eta 0:24:05
     -------------------------------------- 0.1/317.0 MB 554.9 kB/s eta 0:09:31
     ---------------------------------------- 0.6/317.0 MB 2.2 MB/s eta 0:02:24
     ---------------------------------------- 1.1/317.0 MB 3.7 MB/s eta 0:01:26
     ---------------------------------------- 1.6/317.0 MB 4.8 MB/s eta 0:01:07
     ---------------------------------------- 2.2/317.0 MB 5.6 MB/s eta 0:00:57
     ---------------------------------------- 2.7/317.0 MB 6.3 MB/s eta 0:00:51
     ---------------------------------------- 3.3/317.0 MB 6.7 MB/s eta 0:00:47
     ---------------------------------------- 3.8/317.0 MB 7.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, expr, when
from pyspark.sql.types import IntegerType

In [3]:
spark = SparkSession.builder.master("local[*]").appName("Exercicio Intro").getOrCreate()


In [4]:
df_nomes = spark.read.csv("nomes_aleatorios.txt", header=False, inferSchema=True, sep="\t")
df_nomes = df_nomes.withColumnRenamed("_c0", "Nomes")


In [5]:
df_nomes.printSchema()
df_nomes.show(10, truncate=False)

root
 |-- Nomes: string (nullable = true)

+-----------------+
|Nomes            |
+-----------------+
|Frances Bennet   |
|Jamie Russell    |
|Edward Kistler   |
|Sheila Maurer    |
|Donald Golightly |
|David Gray       |
|Joy Bennett      |
|Paul Kriese      |
|Berniece Ornellas|
|Brian Farrell    |
+-----------------+
only showing top 10 rows



In [6]:
df_nomes = df_nomes.withColumn("Escolaridade", expr("CASE WHEN rand() < 0.33 THEN 'Fundamental' WHEN rand() < 0.66 THEN 'Medio' ELSE 'Superior' END"))


In [7]:
from pyspark.sql.functions import expr

paises = ["Brasil", "Argentina", "Colombia", "Peru", "Chile", "Venezuela", "Uruguai", "Paraguai", "Equador", "Bolivia", "Suriname", "Guiana", "Guiana Francesa"]

paises_str = ', '.join(f'"{pais}"' for pais in paises)

expr_str = f"element_at(arrays_zip(array({paises_str}), array({paises_str})), round(cast(rand()*((size(array({paises_str}))-1)) + 1 as int)))"

df_nomes = df_nomes.withColumn("Pais", expr(expr_str))



In [8]:
df_nomes = df_nomes.withColumn("AnoNascimento", expr("round(rand()*(2010-1945) + 1945)").cast(IntegerType()))

In [9]:
df_select = df_nomes.filter(df_nomes["AnoNascimento"] >= 2000)
df_select.show(10)

+--------------------+------------+--------------------+-------------+
|               Nomes|Escolaridade|                Pais|AnoNascimento|
+--------------------+------------+--------------------+-------------+
|       Brian Farrell|       Medio|  {Bolivia, Bolivia}|         2004|
|         Page Marthe| Fundamental|      {Chile, Chile}|         2008|
|        Charles Hill|       Medio|{Suriname, Suriname}|         2000|
|        Rebecca Snow|       Medio|  {Equador, Equador}|         2000|
|         Jessie Jean| Fundamental|{Suriname, Suriname}|         2006|
|    Flossie Anderson|    Superior|  {Uruguai, Uruguai}|         2006|
|       Milton Dillon|       Medio|    {Guiana, Guiana}|         2002|
|         Milton Rowe| Fundamental|    {Brasil, Brasil}|         2007|
|Christopher Williams|       Medio|  {Uruguai, Uruguai}|         2004|
|     Charles Randall|    Superior|{Colombia, Colombia}|         2009|
+--------------------+------------+--------------------+-------------+
only s

In [10]:
df_nomes.createOrReplaceTempView("pessoas")

In [11]:
millennials_count_df = spark.sql("SELECT COUNT(*) AS count FROM pessoas WHERE AnoNascimento BETWEEN 1980 AND 1994")
millennials_count = millennials_count_df.collect()[0]["count"]
print("Número de pessoas da geração Millennials:", millennials_count)

Número de pessoas da geração Millennials: 2306430


In [12]:
generation_query = """
SELECT Pais,
       SUM(CASE WHEN AnoNascimento BETWEEN 1945 AND 1964 THEN 1 ELSE 0 END) AS BabyBoomers,
       SUM(CASE WHEN AnoNascimento BETWEEN 1965 AND 1979 THEN 1 ELSE 0 END) AS GenerationX,
       SUM(CASE WHEN AnoNascimento BETWEEN 1980 AND 1994 THEN 1 ELSE 0 END) AS Millennials,
       SUM(CASE WHEN AnoNascimento BETWEEN 1995 AND 2015 THEN 1 ELSE 0 END) AS GenerationZ
FROM pessoas
GROUP BY Pais
"""

result_df = spark.sql(generation_query)
result_df.show()

result_df = result_df.orderBy("Pais", "BabyBoomers", "GenerationX", "Millennials", "GenerationZ")
result_df.show()

+--------------------+-----------+-----------+-----------+-----------+
|                Pais|BabyBoomers|GenerationX|Millennials|GenerationZ|
+--------------------+-----------+-----------+-----------+-----------+
|    {Brasil, Brasil}|     249746|     192433|     191303|     199191|
|{Argentina, Argen...|     251275|     192565|     192433|     199005|
|{Paraguai, Paraguai}|     249687|     192588|     191964|     198673|
|{Colombia, Colombia}|     250248|     192242|     192873|     198729|
|  {Equador, Equador}|     249474|     192810|     192500|     198678|
|    {Guiana, Guiana}|     251002|     192604|     191787|     198521|
|{Suriname, Suriname}|     249121|     192064|     191759|     198413|
|        {Peru, Peru}|     248581|     192803|     192931|     198960|
|  {Bolivia, Bolivia}|     249950|     192437|     192120|     198479|
|      {Chile, Chile}|     250040|     192167|     193251|     198702|
|  {Uruguai, Uruguai}|     249197|     192960|     191870|     199199|
|{Vene